In [ ]:
!pip install python-docx pandas openpyxl

In [ ]:
from google.colab import files


print('Sube la plantilla FR-EI-02 (Word)') # 👉 Aquí selecciona Plantilla_datos.xlsx y el Word
uploaded = files.upload()
FR_EI_02 = list(uploaded.keys())[0]


Sube la plantilla FR-EI-02 (Word)


Saving FR-EI-02 ENTREGA DE EQUIPO A CONFORMIDAD Y CONDICIONES DE GARANTÍA (1).docx to FR-EI-02 ENTREGA DE EQUIPO A CONFORMIDAD Y CONDICIONES DE GARANTÍA (1) (1).docx


In [ ]:
# Subir plantilla DATOS
print('Sube la plantilla DATOS')
uploaded = files.upload()
PLANTILLA_DATOS = list(uploaded.keys())[0]

Sube la plantilla DATOS


Saving DATOS CLIENTES.xlsx to DATOS CLIENTES (1).xlsx


In [ ]:
# Subir FORMATO
print('Sube FR-EI-03 PROTOCOLO DE MANTENIMIENTO PREVENTIVO FORMATO')
uploaded = files.upload()
FORMATO_PROTO_MMTO = list(uploaded.keys())[0]

Sube FR-EI-03 PROTOCOLO DE MANTENIMIENTO PREVENTIVO FORMATO


Saving FR-EI-03 PROTOCOLO DE MANTENIMIENTO PREVENTIVO - PROYECTO.xlsx to FR-EI-03 PROTOCOLO DE MANTENIMIENTO PREVENTIVO - PROYECTO.xlsx


In [ ]:
# Subir FORMATO
print('Sube FR-EI-04 HOJA DE VIDA DEL EQUIPO  FORMATO')
uploaded = files.upload()
FORMATO_HOJA_DE_VIDA = list(uploaded.keys())[0]

Sube FR-EI-04 HOJA DE VIDA DEL EQUIPO  FORMATO


Saving FR-EI-04 HOJA DE VIDA DEL EQUIPO - PROYECTO.xlsx to FR-EI-04 HOJA DE VIDA DEL EQUIPO - PROYECTO (1).xlsx


In [ ]:
# Subir FORMATO
print('Sube FR-EI-05 CRONOGRAMA DE MANTENIMIENTO FORMATO')
uploaded = files.upload()
CRONOGRAMA_MANTENIMIENTO = list(uploaded.keys())[0]

Sube FR-EI-05 CRONOGRAMA DE MANTENIMIENTO FORMATO


Saving FR-EI-05 CRONOGRAMA DE MANTENIMIENTO - AKR300.xlsx to FR-EI-05 CRONOGRAMA DE MANTENIMIENTO - AKR300 (2).xlsx


In [ ]:
!pip install python-docx pandas openpyxl
from google.colab import files
import pandas as pd
from docx import Document
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os, shutil, zipfile

# ==============================
# 1. Subir archivos base (descomenta en Colab si quieres subir manualmente)
# ==============================
# uploaded = files.upload(); PLANTILLA_DATOS = list(uploaded.keys())[0]
# uploaded = files.upload(); FR_EI_02 = list(uploaded.keys())[0]
# uploaded = files.upload(); FORMATO_HOJA_DE_VIDA = list(uploaded.keys())[0]
# uploaded = files.upload(); FORMATO_PROTO_MMTO = list(uploaded.keys())[0]
# uploaded = files.upload(); CRONOGRAMA_MANTENIMIENTO = list(uploaded.keys())[0]

# ==============================
# 2. Leer los datos
# ==============================
df = pd.read_excel(PLANTILLA_DATOS)

# Crear carpeta de salida
output_dir = "DOCUMENTACION CLIENTES"
os.makedirs(output_dir, exist_ok=True)

# ==============================
# Función auxiliar: escribir respetando merges (sin descombinar)
# ==============================
def write_respecting_merged(ws, target_cell, value):
    """
    Si target_cell pertenece a un rango mergeado, escribe en la celda superior-izquierda
    del rango. Si no, escribe en target_cell directamente.
    """
    for merged in ws.merged_cells.ranges:
        # merged como 'A1:C1' -> str(merged) contiene target_cell si está dentro
        if target_cell in str(merged):
            # obtener coordenadas top-left del rango
            tl_col = get_column_letter(merged.min_col)
            tl_row = merged.min_row
            top_left = f"{tl_col}{tl_row}"
            ws[top_left] = value
            return
    # Si no estaba mergeado, escribir directamente
    ws[target_cell] = value

# ==============================
# 3. Iterar sobre cada fila
# ==============================
for index, row in df.iterrows():
    cliente = str(row["CLIENTE"]).strip().replace(" ", "_")
    equipo  = str(row["NOMBRE DEL EQUIPO"]).strip().replace(" ", "_")
    referencia = str(row["REFERENCIA"]).strip() if pd.notna(row["REFERENCIA"]) else ""
    serie      = str(row["SERIE"]).strip() if pd.notna(row["SERIE"]) else ""
    fecha_word = str(row["FECHA INSTALACION(WORD)"]).strip() if pd.notna(row["FECHA INSTALACION(WORD)"]) else ""
    nit_cliente = str(row["NIT CLIENTE"]).strip() if pd.notna(row["NIT CLIENTE"]) else ""
    tipo_mmto = str(row["TIPO MMTO"]).strip() if pd.notna(row["TIPO MMTO"]) else ""
    FRECUENCIA = str(row["FRECUENCIA"]).strip() if pd.notna(row["FRECUENCIA"]) else ""
    DIRECCION = str(row["DIRECCION"]).strip() if pd.notna(row["DIRECCION"]) else ""
    modelo = str(row["MODELO"]).strip() if pd.notna(row["MODELO"]) else ""
    UBICACION_DEL_EQUIPO = str(row["UBICACIÓN DEL EQUIPO (ÁREA)"]).strip() if pd.notna(row["UBICACIÓN DEL EQUIPO (ÁREA)"]) else ""
    DIRECCION = str(row["DIRECCION"]).strip() if pd.notna(row["DIRECCION"]) else ""

    # Fechas separadas
    dia  = str(row["DD"]).strip() if pd.notna(row["DD"]) else ""
    mes  = str(row["MM"]).strip() if pd.notna(row["MM"]) else ""
    anio = str(row["AA"]).strip() if pd.notna(row["AA"]) else ""

    entidad  = str(row["ENTIDAD"]).strip() if pd.notna(row["ENTIDAD"]) else ""
    ciudad   = str(row["CIUDAD"]).strip() if pd.notna(row["CIUDAD"]) else ""
    telefono = str(row["TELEFONO CLIENTE"]).strip() if pd.notna(row["TELEFONO CLIENTE"]) else ""

    # Carpeta temporal por cliente/equipo
    folder_name = f"{cliente}_{equipo}"
    folder_path = os.path.join(output_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # ==============================
    # A) Generar Word
    # ==============================
    doc = Document(FR_EI_02)

    # Reemplazar cliente en texto
    for p in doc.paragraphs:
        if "(Nombre cliente)" in p.text:
            p.text = p.text.replace("(Nombre cliente)", row["CLIENTE"])

    # Llenar la tabla del Word (ajusta índices si tu tabla es distinta)
    table = doc.tables[0]
    table.cell(0,1).text = str(row["NOMBRE DEL EQUIPO"])
    table.cell(1,1).text = referencia
    table.cell(2,1).text = serie
    table.cell(3,1).text = fecha_word

    word_path = os.path.join(folder_path, f"FR-EI-02 ENTREGA DE EQUIPO A CONFORMIDAD Y CONDICIONES DE GARANTÍA-{cliente}-{equipo}.docx")
    doc.save(word_path)

    # ==============================
    # B) Generar Excel FR-EI-04 HOJA DE VIDA DEL EQUIPO - PROYECTO
    # ==============================
    wb_hv = load_workbook(FORMATO_HOJA_DE_VIDA)
    ws_hv = wb_hv.active

    ws_hv["D9"]  = row["NOMBRE DEL EQUIPO"]
    ws_hv["V24"] = dia
    ws_hv["X24"] = mes
    ws_hv["Y24"] = anio
    ws_hv["D22"]  = entidad
    ws_hv["AE22"] = ciudad
    ws_hv["AE24"] = telefono
    ws_hv["AD7"]  = serie

    excel_hv_path = os.path.join(folder_path, f"FR-EI-04 HOJA DE VIDA DEL EQUIPO-{cliente}-{equipo}.xlsx")
    wb_hv.save(excel_hv_path)
    # ==============================
    # C) Generar Excel FR-EI-03 PROTOCOLO DE MANTENIMIENTO PREVENTIVO - PROYECTO
    # ==============================
    wb_mmttopreventivo = load_workbook(FORMATO_PROTO_MMTO)
    ws_mmttopreventivo = wb_mmttopreventivo.active
    ws_mmttopreventivo["A12"] = row["NOMBRE DEL EQUIPO"]
    excel_ws_mmttopreventivo_path = os.path.join(folder_path, f"FR-EI-03 PROTOCOLO DE MANTENIMIENTO PREVENTIVO-{cliente}-{equipo}.xlsx")
    wb_mmttopreventivo.save(excel_ws_mmttopreventivo_path)
    # ==============================
    # D) Generar Excel CRONOGRAMA MANTENIMIENTO (escribir sin descombinar)
    # ==============================
    wb_cronograma = load_workbook(CRONOGRAMA_MANTENIMIENTO)
    ws_cronograma = wb_cronograma.active
    wb_cronograma = load_workbook(CRONOGRAMA_MANTENIMIENTO)
    ws_cronograma = wb_cronograma.active
    ws_cronograma["B10"]  = row["NOMBRE DEL EQUIPO"]
    ws_cronograma["E10"] = serie
    ws_cronograma["F10"] = tipo_mmto
    ws_cronograma["D5"]  = nit_cliente
    ws_cronograma["R6"]  = fecha_word
    ws_cronograma["G10"] = FRECUENCIA
    ws_cronograma["F5"]  = DIRECCION
    ws_cronograma["R5"]  = ciudad
    ws_cronograma["D10"]  = modelo
    ws_cronograma["D6"]  = UBICACION_DEL_EQUIPO
    ws_cronograma["D4"]  = cliente
    ws_cronograma["R4"]  = row["TELEFONO CLIENTE"]
    excel_cronograma_path= os.path.join(folder_path, f"FR-EI-05 CRONOGRAMA DE MANTENIMIENTO-{cliente}-{equipo}.xlsx")
    wb_cronograma.save(excel_cronograma_path)

    zip_path = os.path.join(output_dir, f"{cliente}_{equipo}.zip")
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for f in os.listdir(folder_path):
            zipf.write(os.path.join(folder_path, f), arcname=f)

    # Limpiar carpeta temporal
    shutil.rmtree(folder_path)

print("✅ Proceso terminado. Los ZIP están en 'DOCUMENTACION CLIENTES'")
shutil.make_archive("DOCUMENTACION CLIENTES", "zip", "DOCUMENTACION CLIENTES")
files.download("DOCUMENTACION CLIENTES.zip")

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


✅ Proceso terminado. Los ZIP están en 'DOCUMENTACION CLIENTES'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>